In [28]:
import numpy as np
np.random.seed(5)

# Permite crear celda recurrente
from keras.layers import Input, Dense, SimpleRNN 
from keras.models import Model
# Crear y entrenar el modelo
from keras.optimizers import SGD 
# Formato
from keras.utils import to_categorical 
from keras import backend as K

Using TensorFlow backend.


In [33]:
nombres = open('dinosaurios.txt').read()
nombres = nombres.lower()
nombres = nombres.replace('\r','\\r').replace('\n',' ')
nombres = nombres.replace('}','')

In [34]:
alfabeto = list(set(nombres))
tam_datos, tam_alfabeto = len(nombres), len(alfabeto)

Diccionario en el que se asigna a cada caracter del alfabeto un número (codificación)

In [35]:
car_a_ind = { car:ind for ind,car in enumerate(sorted(alfabeto))}

In [36]:
ind_a_car = { ind:car for ind,car in enumerate(sorted(alfabeto))}

## Modelo RNN
A continuación se implementara un RNN

In [37]:
n_a = 25
# Contenedores
entrada = Input(shape=(None,tam_alfabeto))
# Estado oculto
a0 = Input(shape=(n_a,))

In [38]:
celdas_recurrentes = SimpleRNN(n_a, activation='tanh', return_state = True )

In [39]:
capa_salida = Dense(tam_alfabeto, activation='softmax')

In [41]:
hs, _ = celdas_recurrentes(entrada, initial_state=a0)
salida = []
salida.append(capa_salida(hs))
modelo = Model([entrada,a0], salida)

opt = SGD(lr=0.05)
modelo.compile(optimizer=opt, loss = 'categorical_crossentropy')

In [50]:
with open('dinosaurios.txt') as f:
    ejemplos = f.readlines()
ejemplos = list(map( lambda x: x.replace("\n",""), ejemplos))
ejemplos = [x.lower().strip() for x in ejemplos]
np.random.shuffle(ejemplos)

In [ ]:
def train_generator():
    while True:
        ejemplo = ejemplos[np.random.randint(0,len(ejemplos))]
        
        X = [None] + [car_a_ind[c] for c in ejemplo]
        Y = X[1:]  + [car_a_ind['\n']]
        
        x = np.zeros((len(X), 1, tam_alfabeto))
        onehot = to_categorical(Y,tam_alfabeto).reshape(len(X),tam_alfabeto)
        
        a = np.zeros((len(X), n_a))
        
        yield [x, a] , y

In [51]:
BATCH_SIZE = 80
NITS = 10000

for j in range(NITS):
    historia = modelo.fit_generator(train_generator(),
                                   steps_per_epoch=BATCH_SIZE, epochs=1, verbose=0)
    if j%1000 == 0:
        print('\n Iteracción: %d, Error: %f' % 
             (j, historia.history['loss'][0] + '\n'))

In [ ]:
def generar_nombre(modelo, car_a_ind, tam_alfabeto,n_a):
    x = np.zeros((1,1,tam_alfabetom,))
    a = np.zeros((1,n_a))
    
    nombre_generado = ''
    fin_linea = '\n'
    car = -1
    contador = 0

